In [ ]:
%pip install -q tqdm numpy pandas scikit_learn tensorflow matplotlib
%matplotlib inline

In [ ]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_labels = []
train_samples =[]

Example Data:

- An experimental drug was tested on individuals from ages 13-100 in a clinical trial
- The trial had 2100 participants. Half were under 65 years old and half were 65 or older
- around 95% of patients or older experienced side effects
- around 95% of patients under 65 experimented no side effects

In [ ]:
for i in range(50):
	# The ~5% of younger individuals who did experience side effects
		random_younger = randint(13, 64)
		train_samples.append(random_younger)
		train_labels.append(1)
		
		# The ~5% of older individuals who did not experience side effects
		random_older = randint(65, 100)
		train_samples.append(random_older)
		train_labels.append(0)
  
for i in range(1000):
		# The ~95% of younger individuals who did not experience side effects
		random_younger = randint(13, 64)
		train_samples.append(random_younger)
		train_labels.append(0)
		
		# The ~95% of older individuals who did experience side effects
		random_older = randint(65, 100)
		train_samples.append(random_older)
		train_labels.append(1)

In [ ]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels, train_samples

In [ ]:
train_labels, train_samples = shuffle(train_labels, train_samples)
train_labels, train_samples

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1, 1))
scaled_train_samples

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
if len(physical_devices) > 0:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
model = Sequential([
	Dense(units=16, input_shape=(1,), activation='relu'),
 	Dense(units=32, activation='relu'),
  Dense(units=2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=scaled_train_samples, y=train_labels, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

In [ ]:
model.save('medical_trial_model.keras')

In [ ]:
from keras.models import load_model
new_model = load_model('medical_trial_model.keras')
new_model.summary()

In [ ]:
wgts = new_model.get_weights()
for i in range(len(wgts)):
  print(wgts[i].shape)

In [ ]:
new_model.optimizer
json_string = model.to_json()

In [ ]:
json_string

In [ ]:
from keras.models import model_from_json
model_architecture = model_from_json(json_string)

In [ ]:
model_architecture.summary()

In [ ]:
test_labels = []
test_samples = []

for i in range(10):
    # The 5% of younger individuals who did experience side effects
    random_younger = randint(13, 64)
    test_samples.append(random_younger)
    test_labels.append(1)

    # The 5% of older individuals who did not experience side effects
    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(0)

for i in range(200):
    # The 95% of younger individuals who did not experience side effects
    random_younger = randint(13, 64)
    test_samples.append(random_younger)
    test_labels.append(0)

    # The 95% of older individuals who did experience side effects
    random_older = randint(65, 100)
    test_samples.append(random_older)
    test_labels.append(1)
    
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_test_samples = scaler.fit_transform((test_samples).reshape(-1, 1))

In [ ]:
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['no_side_effects', 'had_side_effects']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')